هو عبارة عن تجميع كل الخطوات اللي سويناها (EDA → Cleaning → Splitting → Balancing→ Preprocessing) في تسلسل واضح ومرتب

Imports & Config

In [1]:
from pathlib import Path
from PIL import Image
import random, json
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np

In [2]:
# مسار الداتا بعد التقسيم
SPLIT_DIR = Path("./ArASL_Split")

# نستخدم قناة وحدة (رمادي). خليها True لو بتستخدمين موديلات pretrained (3 قنوات)
USE_RGB_3CH = False

# مقاس الإدخال (متوافق مع التنظيف السابق)
IMG_SIZE   = (64, 64)
BATCH_SIZE = 128
NUM_WORKERS = 4
PIN_MEMORY = True and torch.cuda.is_available()

# ثبّت العشوائية
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

Classes mapping

In [3]:
#  نقرأ أسماء الكلاسات من train ونثبّت الترتيب -> label ثابت
#الهدف: نحصل على قائمة الكلاسات وتعريفها بأرقام ثابتة

train_root = SPLIT_DIR / "train"
assert train_root.exists(), f"Train folder not found: {train_root}"

classes = sorted([d.name for d in train_root.iterdir() if d.is_dir()])
class_to_idx = {c: i for i, c in enumerate(classes)}
idx_to_class = {i: c for c, i in class_to_idx.items()}
print(f"[INFO] classes ({len(classes)}):", classes)

[INFO] classes (32): ['ain', 'al', 'aleff', 'bb', 'dal', 'dha', 'dhad', 'fa', 'gaaf', 'ghain', 'ha', 'haa', 'jeem', 'kaaf', 'khaa', 'la', 'laam', 'meem', 'nun', 'ra', 'saad', 'seen', 'sheen', 'ta', 'taa', 'thaa', 'thal', 'toot', 'waw', 'ya', 'yaa', 'zay']


Basic transforms 

In [4]:
# هنا نخلي التحويلات بسيطة ونفسها للـ train/val/test 
def _to_tensor_gray_or_rgb():
    ops = [transforms.ToTensor()]  # يحوّل إلى Tensor بنطاق [0..1] وشكل CxHxW
    if USE_RGB_3CH:
        # لو الصورة 1قناة، نوسّعها إلى 3 قنوات (لموديلات 3ch)
        ops.append(transforms.Lambda(lambda t: t.expand(3, *t.shape[1:])))
    return transforms.Compose(ops)

basic_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),     # تأكيد رمادي
    transforms.Resize(IMG_SIZE, antialias=True),     # تأكيد المقاس
    _to_tensor_gray_or_rgb(),
    # Normalize بسيط حول 0.5. تقدرين لاحقًا تحسبين mean/std من train لو حبيتي.
    transforms.Normalize(mean=[0.5] * (3 if USE_RGB_3CH else 1),
                         std =[0.5] * (3 if USE_RGB_3CH else 1)),
])

train_tf = basic_tf
val_tf   = basic_tf
test_tf  = basic_tf

Dataset

In [5]:
class ArASLDataset(Dataset):
    """
      يقرأ الصور من مجلد split (train/val/test) بشكل:
      root/class_name/*.png → (image_tensor, label_int)
    """
    def __init__(self, root_dir: Path, class_to_idx: dict, transform=None):
        self.root_dir = Path(root_dir)
        self.class_to_idx = class_to_idx
        self.transform = transform

        self.samples = []
        for cls_name, idx in self.class_to_idx.items():
            cls_dir = self.root_dir / cls_name
            if not cls_dir.exists():
                continue
            for p in cls_dir.iterdir():
                if p.suffix.lower() in {".png", ".jpg", ".jpeg", ".bmp"}:
                    self.samples.append((p, idx))

        assert len(self.samples) > 0, f"No images under {self.root_dir}"
        # ترتيب ثابت؛ الشفل يصير في DataLoader
        self.samples.sort(key=lambda x: (x[1], x[0].name))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, i):
        path, label = self.samples[i]
        # NOTE (Arabic): نقرأ بـ PIL (أفضل لـ torchvision.transforms)
        img = Image.open(path)
        # لو الملف أساسًا رمادي/ألوان، Grayscale بالـ transform سيضمن قناة واحدة
        if self.transform:
            img = self.transform(img)
        return img, label

DataLoaders

In [6]:
train_ds = ArASLDataset(SPLIT_DIR/"train", class_to_idx, transform=train_tf)
val_ds   = ArASLDataset(SPLIT_DIR/"val",   class_to_idx, transform=val_tf)
test_ds  = ArASLDataset(SPLIT_DIR/"test",  class_to_idx, transform=test_tf)

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
)
val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
)
test_loader = DataLoader(
    test_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
)

print(f"[INFO] train/val/test sizes = {len(train_ds)} / {len(val_ds)} / {len(test_ds)}")

[INFO] train/val/test sizes = 48038 / 4842 / 4842
